In [1]:
import math, hashlib, os.path

## System Consts

In [2]:
mu = 0.1
delta = 1e-3
cToC = 0.3
L = 1.0
sigma = 0.1
beta_naught = math.pi / 180.0 * 23 / 2

config_string = "{}-{}-{}-{}-{}-{}".format(mu, delta, cToC, L, sigma, beta_naught)
config_hash = hashlib.sha1(bytes(config_string, 'ascii')).hexdigest()

config_root = "./config/"

## Data Gen

In [3]:
import cuda_cmm as cuda_cmm
import numpy as np

if (not os.path.exists("{}cmm_bauxite-{}.npy".format(config_root, config_hash))):
    num_alphas = 2<<6


    print("\nBauxite Data Generation")
    # 2.3578
    # 3.9254 
    alphas = np.linspace(0.7*np.pi, 1/0.7*np.pi, num_alphas, dtype=np.float32)

    num_combos = num_alphas
    print("num_combos:", num_combos)


    vec = np.zeros((num_combos, 4), dtype=np.float32)

    # print("Starting Vector loading")
    ind = 0
    for k in range(num_alphas):
        ind = k
        vec[ind, 0:3] = [1e9, -1e6, alphas[k]]

    print("Starting Cuda")
    cuda_cmm.fr(vec, beta_naught, mu)

    print("\nCuda Done")

    np.save("{}cmm_bauxite-{}.npy".format(config_root, config_hash), vec)

if (not os.path.exists("{}cmm_wolframite-{}.npy".format(config_root, config_hash))):
    print("\nWolframite Data Generation")

    num_frs = 2<<9
    num_alphas = 2<<9

    frs = np.linspace(-1.1, 1.1, num_frs, dtype=np.float32)
    alphas = np.linspace(0.7*np.pi, 1/0.7*np.pi, num_alphas, dtype=np.float32)

    num_combos = num_frs*num_alphas
    print("num_combos:", num_combos)


    vec = np.zeros((num_combos, 3), dtype=np.float32)

    # print("Starting Vector loading")
    ind = 0
    for i, fr in enumerate(frs):
        for j, alpha in enumerate(alphas):
                ind = i * num_alphas + j
                vec[ind, 0:2] = [fr, alpha]

    print("Starting Cuda")
    cuda_cmm.eq_clamp(vec, beta_naught, mu, delta)
    print("\nCuda Done")

    np.save("{}cmm_wolframite-{}.npy".format(config_root, config_hash), vec)

if (not os.path.exists("{}cmm_magnetite-{}.npy".format(config_root, config_hash))):
    print("\nMagnetite Data Generation")
    # float fr = vec[5 * i + 0];
    # float A = vec[5 * i + 1];
    # float tau = vec[5 * i + 2];

    num_frs = 2<<6
    num_As = 2<<8
    num_taus = 2<<6

    frs = np.linspace(-1.1, 1.1, num_frs, dtype=np.float32)
    As = np.linspace(-50, 50, num_As, dtype=np.float32)
    taus = np.linspace(0.5, 2.0, num_taus, dtype=np.float32)

    num_combos = num_frs*num_As*num_taus
    print("num_combos:", num_combos)


    vec = np.zeros((num_combos, 6), dtype=np.float32)

    # print("Starting Vector loading")

    for i, fr in enumerate(frs):
        for j, A in enumerate(As):
            for k, tau in enumerate(taus):
                ind = i * num_As * num_taus + j * num_taus  + k
                vec[ind, 0:3] = [fr, A, tau]

    print("Starting Cuda")
    cuda_cmm.c_coefficient(vec, beta_naught, mu, delta, cToC, L, sigma)
    # float beta_naught, float mu, float delta, float cToC, float L, float sigma

    print("\nCuda Done")

    np.save("{}cmm_magnetite-{}.npy".format(config_root, config_hash), vec)


# Network Definition

In [4]:
HIDDEN = 512

network_config_string = "{}-{}-{}-{}-{}-{}-{}".format(mu, delta, cToC, L, sigma, beta_naught, HIDDEN)
network_config_hash = hashlib.sha1(bytes(network_config_string, 'ascii')).hexdigest()

In [5]:
import torch

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class BauxiteNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = torch.nn.Sequential(
            torch.nn.Linear(1, HIDDEN),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(HIDDEN, HIDDEN),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(HIDDEN, HIDDEN),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(HIDDEN, 1),
        )

    def forward(self, x):
        logits = self.stack(x)
        return logits

# Define model
class WolframiteNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = torch.nn.Sequential(
            torch.nn.Linear(2, HIDDEN),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(HIDDEN, HIDDEN),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(HIDDEN, HIDDEN),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(HIDDEN, 1),
        )

    def forward(self, x):
        logits = self.stack(x)
        return logits

# Define model
class MagnetiteNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = torch.nn.Sequential(
            torch.nn.Linear(3, HIDDEN),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(HIDDEN, HIDDEN),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(HIDDEN, HIDDEN),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(HIDDEN, 3),
        )

    def forward(self, x):
        logits = self.stack(x)
        return logits

Using cuda device


In [6]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class BauxiteDataset(Dataset):
    def __init__(self, data_file, transform=None, target_transform=None):
        raw_data = np.load(data_file) # [vel, A, alpha, fr]

        self.data = torch.tensor(raw_data[np.isfinite(raw_data[:, 2]) & np.isfinite(raw_data[:, 3]), 2:4], dtype=torch.float) # [alpha, fr]

        print(raw_data.shape[0] - self.data.shape[0], 'samples removed due to NaNs')

        # print(self.data[~torch.isfinite(self.data).all(1)])

        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        state = self.data[idx, 0:1] # [alpha]
        label = self.data[idx, 1:2] # [fr]
        if self.transform:
            state = self.transform(state)
        if self.target_transform:
            label = self.target_transform(label)
        return state, label

class WolframiteDataset(Dataset):
    def __init__(self, data_file, transform=None, target_transform=None):
        raw_data = np.load(data_file) # [fr, alpha, eq_clamp]

        self.data = torch.tensor(raw_data[np.isfinite(raw_data[:, 2])], dtype=torch.float) # [fr, alpha, eq_clamp]

        print(raw_data.shape[0] - self.data.shape[0], 'samples removed due to NaNs')

        # print(self.data[~torch.isfinite(self.data).all(1)])

        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        state = self.data[idx, 0:2] # [fr, alpha]
        label = self.data[idx, 2:] # [eq_clamp]
        if self.transform:
            state = self.transform(state)
        if self.target_transform:
            label = self.target_transform(label)
        return state, label

class MagnetiteDataset(Dataset):
    def __init__(self, data_file, transform=None, target_transform=None):
        raw_data = np.load(data_file) # [fr, A, tau, clamp_ratio, tau_effec, dim_clamp_prim]

        clean_data = raw_data[np.isfinite(raw_data[:, 4])]

        self.data = torch.tensor(np.c_[clean_data[:, 0], clean_data[:, 2], clean_data[:, 3], clean_data[:, 1], clean_data[:, 4], clean_data[:, 5]], dtype=torch.float) # [fr, tau, clamp_ratio, A, tau_effec, dim_clamp_prim]


        print(raw_data.shape[0] - self.data.shape[0], 'samples removed due to NaNs')

        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        state = self.data[idx, 0:3] # [fr, tau, clamp_ratio]
        label = self.data[idx, 3:] # [A, tau_effec, Q]
        if self.transform:
            state = self.transform(state)
        if self.target_transform:
            label = self.target_transform(label)
        return state, label

In [7]:
BATCH_SIZE = 2048

def train(dataloader, model, loss_fn, optimizer):
            size = len(dataloader.dataset)
            model.train()
            for batch, (X, y) in enumerate(dataloader):
                X, y = X.to(device).float(), y.to(device).float()

                # Compute prediction error
                pred = model(X)
                loss = loss_fn(pred, y)

                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

def test(dataloader, model, loss_fn, check_fn):
    # size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    check_val = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device).float()
            pred = model(X)

            test_loss += loss_fn(pred, y).item()
            check_val += check_fn(pred, y).item()
    test_loss /= num_batches
    check_val /= num_batches

    return test_loss, check_val


for model, name, dataset in [(BauxiteNetwork(), "bauxite", BauxiteDataset("{}cmm_bauxite-{}.npy".format(config_root, config_hash))), (WolframiteNetwork(), "wolframite", WolframiteDataset("{}cmm_wolframite-{}.npy".format(config_root, config_hash))), (MagnetiteNetwork(), "magnetite", MagnetiteDataset("{}cmm_magnetite-{}.npy".format(config_root, config_hash)))]:
    if (not os.path.exists("{}cmm_{}-{}.pth".format(config_root, name, network_config_hash))):
        print("Training {} network".format(name))
        model = model.to(device)

        loss_fn = torch.nn.MSELoss()
        check_fn = torch.nn.L1Loss()

        lr = 0.9e-6
        optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
        import time

        cmm_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
        last_loss = 30.0

        epochs = 50
        epochs = 10
        for t in range(epochs):
            start = time.perf_counter()
            if t % 10 == 0:
                lr /= 8
                optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
                # print("\nlr = ", lr)


            train(cmm_dataloader, model, loss_fn, optimizer)
            test_loss, check_val = test(cmm_dataloader, model, loss_fn, check_fn)
            print(f"Epoch {t+1}", (f"Old loss: {last_loss:>8f} New loss: {test_loss:>8f} Check fn: {check_val:>8f}"), "Took {:.3f}".format(time.perf_counter() - start), end="\r")

            last_loss = test_loss

        print("Done!")
        torch.save(model.state_dict(), "{}cmm_{}-{}.pth".format(config_root, name, network_config_hash))

0 samples removed due to NaNs
0 samples removed due to NaNs
0 samples removed due to NaNs


In [8]:
bauxite = BauxiteNetwork()
bauxite.load_state_dict(torch.load("{}cmm_bauxite-{}.pth".format(config_root, network_config_hash)))
bauxite.to(device).eval()

wolframite = WolframiteNetwork()
wolframite.load_state_dict(torch.load("{}cmm_wolframite-{}.pth".format(config_root, network_config_hash)))
wolframite.to(device).eval()

magnetite = MagnetiteNetwork()
magnetite.load_state_dict(torch.load("{}cmm_magnetite-{}.pth".format(config_root, network_config_hash)))
magnetite.to(device).eval()

print(bauxite)
print(wolframite)
print(magnetite)

BauxiteNetwork(
  (stack): Sequential(
    (0): Linear(in_features=1, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
)
WolframiteNetwork(
  (stack): Sequential(
    (0): Linear(in_features=2, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
)
MagnetiteNetwork(
  (stack): Sequential(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_f

# Engine Loading

In [9]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import csv
import numpy as np

ENGINE = "BRIGGS"
# ENGINE = "KHOLER"

filename = ""
if ENGINE == "BRIGGS":
    filename = "BriggsModel19Data.csv"
elif ENGINE == "KHOLER":
    filename = "RestrictedKohlerCH440.csv"

data = []

with open(filename, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    spamreader.__next__() # Skip header
    for row in spamreader:
        rpm = int(row[0])

        omega = rpm / 60 * 2 * np.pi # rpm to rad/s
        power = float(row[1]) * 745.7 # hp to W
        torque = float(row[2]) * 1.35582 # Lb-Ft to N-m

        data.append((omega, power, torque))

data_ray = np.array(data)
pow_ray = data_ray[:, [0, 1]]
tor_ray = data_ray[:, [0, 2]]

# max_speed = data_ray[-1][0]

def ndd(ray):
    if ray.shape[0] == 2:
        return (ray[1][1] - ray[0][1]) / (ray[1][0] - ray[0][0])
    else:
        return (ndd(ray[1:]) - ndd(ray[:-1])) / (ray[-1][0] - ray[0][0])


# test_ray = np.asarray([[0.0, 10], [1.0, 20], [2.0, 10]])
# test_ray
# compute nddp of torque
tor_coeffs = np.zeros((data_ray.shape[0]))


tor_coeffs[0] = tor_ray[0][1]
for i in range(1, data_ray.shape[0]):
    tor_coeffs[i] = ndd(tor_ray[0: i + 1])

# print(tor_coeffs)

def tor_nddp(omega):
    y = tor_coeffs[0] * omega ** 0
    
    prod = omega ** 0
    for i in range(1, tor_coeffs.shape[0]):
        prod *= omega - tor_ray[i - 1][0]
        y += tor_coeffs[i] * prod
    
    return y

# Check the NDDP at all the points
for omega, torque in tor_ray:
    print(omega, tor_nddp(omega), torque)


209.43951023931956 18.642525 18.642525
230.3834612632515 18.710316000000002 18.710316000000002
251.32741228718345 18.913688999999998 18.913688999999998
272.2713633111154 18.98148 18.98148
293.21531433504737 18.913688999999998 18.913688999999998
314.1592653589793 18.845898000000002 18.845898000000002
335.1032163829113 18.642525 18.642525
356.0471674068432 18.303569999999997 18.30357
376.99111843077515 17.62566 17.62566


In [11]:
print(397.9351, tor_nddp(397.9351))

397.9351 10.914327939019799


# Run the Model

In [12]:
def rand_range(lower, upper, rand):
    return lower + (upper - lower) * rand

In [13]:
SIMS = 16384 // 1024
print(SIMS)
# SIMS = 1
ITERS = 600

NEWTON_STEPS = 100
NEWTON_THRESH = 1e-6

with torch.no_grad():

    def solveForOther(radius, cToC, L):
            def L_prime(other_radius):
                theta_t = torch.asin((other_radius - radius) / cToC)

                alpha_radius = torch.pi - 2 * theta_t
                alpha_other_radius = torch.pi + 2 * theta_t
                length = radius * alpha_radius + other_radius * alpha_other_radius + 2 * torch.sqrt(cToC**2 - (other_radius - radius)**2)

                return length

            def dL_dOther(other_radius):
                theta_t = torch.asin((other_radius - radius) / cToC)
                alpha_other_radius = torch.pi + 2 * theta_t

                dthetaT = 1 / torch.sqrt(1 - (other_radius - radius) / cToC) * (1 / cToC)

                dalpha_other = 2 * dthetaT

                dsl_dOther = 0.5 * (cToC**2 - (other_radius - radius)**2)**(-0.5) * (-2 * (other_radius - radius))

                delta_length = alpha_other_radius + other_radius * dalpha_other + 2 * dsl_dOther

                return delta_length

            # Newtons Method to find Radii that fit the speed ratio and belt length
            guess = radius.clone()
            iter = 0
            discrep = torch.ones(SIMS, 1, device=device, requires_grad=False)
            while (torch.abs(discrep) > NEWTON_THRESH).any() and iter < NEWTON_STEPS:
                discrep = L_prime(guess) - L
                # print("Iter:", iter, "guess:", guess, "discrep:", discrep, "slope:", dL_dOther(guess))
                guess -= discrep / dL_dOther(guess)

                iter += 1

            return guess

    def dLdRp(r_p, r_s, cToC):
        return torch.pi - 2 * torch.atan((r_s - r_p)/cToC) - 2/cToC * (r_s - r_p) / (1 + ((r_s - r_p)/cToC)**2) - 2 * r_p / (cToC**2 + (r_s - r_p)**2).sqrt()

    def dLdRs(r_p, r_s, cToC):
        return torch.pi + 2 * torch.atan((r_s - r_p)/cToC) + 2/cToC * (r_s - r_p) / (1 + ((r_s - r_p)/cToC)**2) + 2 * r_s / (cToC**2 + (r_s - r_p)**2).sqrt()
    
    DELTA_T = 0.001 * torch.ones(SIMS, 1, device=device)


    beta_naught = math.pi / 180.0 * 23 / 2
    mu = 0.1
    sheave_terms = (1 + math.cos(beta_naught)**2) / math.sin(2 * beta_naught)
    delta = 1e-3

    # Engine Config
    omega = 2000/60*2*math.pi * torch.ones(SIMS, 1, device=device, requires_grad=False)
    throttle = 1.0
    engine_inertia = 0.1 * 0.5 # kg * m^2

    # CVT Config
    cToC = 0.350
    L = 1.8
    sigma = 0.1 # kg / m
    natural_tension = torch.zeros(SIMS, 1, device=device, requires_grad=False)

    r_p = torch.ones(SIMS, 1, device=device, requires_grad=False) * 0.1

    r_s = solveForOther(r_p, cToC, L)

    tau_time = np.zeros((ITERS, 2))

    # Clamp Config
    o_gain = rand_range(0.0, 2.0, torch.rand(SIMS, 1, device=device)) # N * sec / rad
    rad_gain = rand_range(0.0, 2.0, torch.rand(SIMS, 1, device=device)) # N

    rad_gain_s = rand_range(0.0, 2.0, torch.rand(SIMS, 1, device=device)) # N
    torque_gain_s = rand_range(-2.0, 0.0, torch.rand(SIMS, 1, device=device)) # 1 / m

    # Car Config
    mass = 150 # kg
    wheel_radius = 0.3 # m
    with open('gearbox.txt', 'r') as file:
        ratio_string = file.read()
    gearbox_ratio = eval(ratio_string)

    # Sim Config
    x = torch.zeros(SIMS, 1, device=device, requires_grad=False)
    x_dot = torch.zeros(SIMS, 1, device=device, requires_grad=False)
    tau_effective = torch.zeros(SIMS, 1, device=device, requires_grad=False)
    omega_belt = torch.zeros(SIMS, 1, device=device, requires_grad=False)
    torque_max = torch.zeros(SIMS, 1, device=device, requires_grad=False)


    dist = 30.48 # m

    t = torch.ones(SIMS, 1, device=device, requires_grad=False) * -0.5

    brakes = torch.ones(SIMS, 1, device=device, requires_grad=False)

    i = 0
    while i < ITERS and (x < dist).any():
        brakes = t < 0.0

        omega = torch.clamp(omega, min=2000/60*2*math.pi, max=3800/60*2*math.pi)

        
        if ~torch.isfinite(omega).any():
            print("Omega is not finite")
            break

        r_p = torch.clamp(r_p, min=0.01, max=0.24)
        r_s = solveForOther(r_p, cToC, L)

        if ~torch.isfinite(r_p).any():
            print("r_p is not finite")
            break

        if ~torch.isfinite(r_s).any():
            print("r_s is not finite")
            break


        tau = r_p / r_s
        omega_belt = x_dot / (wheel_radius * gearbox_ratio * tau)
        # tau_time[i] = [t[82, 0].cpu(), tau[82, 0].cpu()]

        torque = tor_nddp(omega)
        if ~torch.isfinite(torque).any():
            print("Torque is not finite")
            break

        prim = torch.clamp(omega * o_gain + r_p * rad_gain, min=1e-4)

        sec = torch.clamp(r_s * rad_gain_s + torque / tau * torque_gain_s, min=1e-4)

        natural_tension = (prim + sec) / math.tan(beta_naught)

        # print("Natural Tension: {:.5f}".format(natural_tension[0, 0]))
        
        inert = sigma * torch.pow(omega_belt, 2) * torch.pow(r_p, 2)

        fr = torch.log((natural_tension - torque / (2 * r_p) - inert) / (natural_tension + torque / (2 * r_p) - inert))

        fr = torch.isfinite(fr) * fr

        theta_t = torch.asin((r_s - r_p) / cToC)
        alpha_p = torch.pi - 2 * theta_t
        alpha_s = torch.pi + 2 * theta_t

        fr_bound = bauxite(alpha_p)
        fr = torch.isfinite(fr) * fr
        fr = torch.clamp(fr, min=-0.95*fr_bound, max=0.95*fr_bound)

        wolframite_p = wolframite(torch.cat((fr, alpha_p), dim=1))
        wolframite_s = wolframite(torch.cat((-fr, alpha_s), dim=1))

        clamp_ratio = torch.log(prim / sec * wolframite_s / wolframite_p)

        magnetite_p = magnetite(torch.cat((fr, tau, clamp_ratio), dim=1))

        # print(clamp_ratio)
        # print(magnetite_p)

        F_naught = prim / magnetite_p[:, 2:3] + inert

        torque_max = torch.abs(torch.exp(fr) - 1) * r_p * torch.clamp((F_naught - inert), min=0.0)

        if ~torch.isfinite(fr).any():
            print("fr is not finite")
            break

        A_p = magnetite_p[:, 0:1]
        tau_effective = magnetite_p[:, 1:2]
        if ~torch.isfinite(A_p).any():
            print("A_p is not finite")
            break

        if ~torch.isfinite(tau_effective).any():
            print("tau_effective is not finite")
            break
        
        if ~torch.isfinite(torque_max).any():
            print("torque_max is not finite")
            break

        if ~torch.isfinite(x).any():
            print("x is not finite")
            break

        if ~torch.isfinite(omega_belt).any():
            print("omega_belt is not finite")
            break

        deltaR_p = A_p * delta * omega_belt * r_p * sheave_terms * torch.isfinite(fr)
        if ~torch.isfinite(deltaR_p).any():
            print("deltaR_p is not finite")
            break
        # deltaR_s = -deltaR_p * dLdRp(r_p, r_s, cToC) / dLdRs(r_p, r_s, cToC)

        # Apply max torque to the car and remainder to the engine
        omega_dot_engine = (torque - torque_max) / engine_inertia
        omega_dot_car = torque_max / (mass * wheel_radius**2 * (gearbox_ratio**(-2)) * tau**2) * (brakes == 0)

        
        # If grip is sufficient match the rot speed of the two
        omega_dot = torque / (engine_inertia + (mass * wheel_radius**2 * (gearbox_ratio**(-2)) * tau**2))
        omega_dot_engine += (omega_dot - omega_dot_engine) * ((omega_dot_car > omega_dot_engine) & (brakes == 0))
        omega_dot_car += (omega_dot - omega_dot_car) * ((omega_dot_car > omega_dot_engine) & (brakes == 0))


        if ~torch.isfinite(omega_dot).any():
            print("Omega dot is not finite")
            break

        DELTA_T += (1e-5 - DELTA_T) * (torch.abs(x - dist) < 0.01)
            
        bitmap = x < dist

        t += DELTA_T * bitmap
        r_p += DELTA_T * deltaR_p * bitmap
        # r_s += DELTA_T * deltaR_s
        omega += DELTA_T * omega_dot_engine * bitmap
        x += (x_dot * DELTA_T + 0.5 * omega_dot_car * DELTA_T**2 * gearbox_ratio * wheel_radius * tau_effective) * bitmap
        x_dot += omega_dot_car * DELTA_T * gearbox_ratio * wheel_radius * tau_effective * bitmap

        print("Iter: {:d} t[-1] {:.3f} x[-1] {:.5f} omega[-1] {:.5f} R_p[-1] {:.5f} R_s[-1] {:.5f} Natural Tension: {:.5f}".format(i + 1, t[-1, 0], x[-1, 0], omega[-1, 0], r_p[-1, 0], r_s[-1, 0], natural_tension[-1, 0]), end="\t\t\t\r")

        i += 1

# Clear out any NaNs
t *= torch.isfinite(x)

%matplotlib inline
import matplotlib.pyplot as plt

# plt.figure()
# plt.scatter(o_gain.cpu(), t.cpu())
# plt.figure()
# plt.scatter(tau_gain.cpu(), t.cpu())
# plt.figure()
# plt.scatter(tau_gain_s.cpu(), t.cpu())
# plt.figure()
# plt.scatter(torque_gain_s.cpu(), t.cpu())
# plt.figure()
# plt.title("Tau vs Time")
# plt.scatter(tau_time[:, 0], tau_time[:, 1])

16
